# **Data Preparation**
---
Prepare the dataset from The DAEWOO Steel Co. Ltd in Gwangyang, South Korea. It produces several types of coils, steel plates, and iron plates. The information on electricity consumption is held in a cloud-based system. The information on energy consumption of the industry is stored on the website of the Korea Electric Power Corporation (pccs.kepco.go.kr), and the perspectives on daily, monthly, and annual data are calculated and shown. Avalaible in www.uci.edu

### **Import package and Data**

In [1]:
import pandas as pd
import numpy as np

import joblib

In [2]:
df = pd.read_csv('Steel_industry_data.csv')
df.head(2)

,date,Usage_kWh,Lagging_Current_Reactive.Power_kVarh,Leading_Current_Reactive_Power_kVarh,CO2(tCO2),Lagging_Current_Power_Factor,Leading_Current_Power_Factor,NSM,WeekStatus,Day_of_week,Load_Type
0,01/01/2018 00:15,3.17,2.95,0.0,0.0,73.21,100.0,900,Weekday,Monday,Light_Load
1,01/01/2018 00:30,4.00,4.46,0.0,0.0,66.77,100.0,1800,Weekday,Monday,Light_Load


### **Checking duplicated**

In [3]:
def checking_data(filename):
    #before check duplicated
    df = filename
    print(f'{df.shape} #Raw #Columns')
    
    #checking duplicated
    df = df.drop_duplicates()
    print(f'{df.shape} #Raw #Columns')
    
    return df

In [4]:
df = checking_data(df)
df.head(2)

(35040, 11) #Raw #Columns
(35040, 11) #Raw #Columns


,date,Usage_kWh,Lagging_Current_Reactive.Power_kVarh,Leading_Current_Reactive_Power_kVarh,CO2(tCO2),Lagging_Current_Power_Factor,Leading_Current_Power_Factor,NSM,WeekStatus,Day_of_week,Load_Type
0,01/01/2018 00:15,3.17,2.95,0.0,0.0,73.21,100.0,900,Weekday,Monday,Light_Load
1,01/01/2018 00:30,4.00,4.46,0.0,0.0,66.77,100.0,1800,Weekday,Monday,Light_Load


### **Checking validation and NA**

**NA check**

In [5]:
df.isna().any()

date                                    False
Usage_kWh                               False
Lagging_Current_Reactive.Power_kVarh    False
Leading_Current_Reactive_Power_kVarh    False
CO2(tCO2)                               False
Lagging_Current_Power_Factor            False
Leading_Current_Power_Factor            False
NSM                                     False
WeekStatus                              False
Day_of_week                             False
Load_Type                               False
dtype: bool

**Cross check data validation**

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 35040 entries, 0 to 35039
Data columns (total 11 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   date                                  35040 non-null  object 
 1   Usage_kWh                             35040 non-null  float64
 2   Lagging_Current_Reactive.Power_kVarh  35040 non-null  float64
 3   Leading_Current_Reactive_Power_kVarh  35040 non-null  float64
 4   CO2(tCO2)                             35040 non-null  float64
 5   Lagging_Current_Power_Factor          35040 non-null  float64
 6   Leading_Current_Power_Factor          35040 non-null  float64
 7   NSM                                   35040 non-null  int64  
 8   WeekStatus                            35040 non-null  object 
 9   Day_of_week                           35040 non-null  object 
 10  Load_Type                             35040 non-null  object 
dtypes: float64(6), 

In [13]:
df['date'] = pd.to_datetime(df['date'])

In [14]:
def get_unique_cat(df):
    unique_values_list = []

    for column in df.columns:
        if df[column].dtype == 'object':
            unique_values = df[column].unique()
            unique_values_list.append(f"{column}: {', '.join(unique_values)}")

    return unique_values_list

In [15]:
df_object_unique = get_unique_cat(df)
df_object_unique

['WeekStatus: Weekday, Weekend',
 'Day_of_week: Monday, Tuesday, Wednesday, Thursday, Friday, Saturday, Sunday',
 'Load_Type: Light_Load, Medium_Load, Maximum_Load']

**NOTE** : The 'Date' column will not be removed from this dataset because it will be used in the exploratory data analysis (EDA). After the EDA is complete, the 'Date' column will be removed

### **Data Definition**

In [16]:
df.columns

Index(['date', 'Usage_kWh', 'Lagging_Current_Reactive.Power_kVarh',
       'Leading_Current_Reactive_Power_kVarh', 'CO2(tCO2)',
       'Lagging_Current_Power_Factor', 'Leading_Current_Power_Factor', 'NSM',
       'WeekStatus', 'Day_of_week', 'Load_Type'],
      dtype='object')

In [21]:
#rename the columns like the paper
df_renamed = df.rename(columns = {'Usage_kWh' : 'Usage',
                                 'Lagging_Current_Reactive.Power_kVarh': 'LagRP',
                                 'Leading_Current_Reactive_Power_kVarh' : 'LeadRP',
                                 'CO2(tCO2)': 'CO2',
                                 'Lagging_Current_Power_Factor': 'LagPF',
                                  'Leading_Current_Power_Factor' : 'LeadPF',
                                 'NSM': 'NSM',
                                 'WeekStatus': 'WStatus',
                                 'Day_of_week': 'Dweek',
                                 'Load Type': 'Ltype'})

In [22]:
df_renamed.head(5)

,date,Usage,LagRP,LeadRP,CO2,LagPF,LeadPF,NSM,WStatus,Dweek,Load_Type
0,2018-01-01 00:15:00,3.17,2.95,0.0,0.0,73.21,100.0,900,Weekday,Monday,Light_Load
1,2018-01-01 00:30:00,4.00,4.46,0.0,0.0,66.77,100.0,1800,Weekday,Monday,Light_Load
2,2018-01-01 00:45:00,3.24,3.28,0.0,0.0,70.28,100.0,2700,Weekday,Monday,Light_Load
3,2018-01-01 01:00:00,3.31,3.56,0.0,0.0,68.09,100.0,3600,Weekday,Monday,Light_Load
4,2018-01-01 01:15:00,3.82,4.50,0.0,0.0,64.72,100.0,4500,Weekday,Monday,Light_Load


### **Spliting and Dumping Data**

In [23]:
def split_input_output(data, become_y):
    
    y = data[become_y]
    X = data.drop(columns = become_y)
    
    return X,y

In [25]:
X, y = split_input_output(df_renamed, 'Usage')
X.head()

,date,LagRP,LeadRP,CO2,LagPF,LeadPF,NSM,WStatus,Dweek,Load_Type
0,2018-01-01 00:15:00,2.95,0.0,0.0,73.21,100.0,900,Weekday,Monday,Light_Load
1,2018-01-01 00:30:00,4.46,0.0,0.0,66.77,100.0,1800,Weekday,Monday,Light_Load
2,2018-01-01 00:45:00,3.28,0.0,0.0,70.28,100.0,2700,Weekday,Monday,Light_Load
3,2018-01-01 01:00:00,3.56,0.0,0.0,68.09,100.0,3600,Weekday,Monday,Light_Load
4,2018-01-01 01:15:00,4.50,0.0,0.0,64.72,100.0,4500,Weekday,Monday,Light_Load


In [26]:
y.head()

0    3.17
1    4.00
2    3.24
3    3.31
4    3.82
Name: Usage, dtype: float64

In [27]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,
                                                   y,
                                                   test_size = 0.2,
                                                   random_state = 123)

In [28]:
# save data
joblib.dump(X_train,"D:/ML/INTRO ML/Project/joblib_train_test/X_train.csv")
joblib.dump(y_train,"D:/ML/INTRO ML/Project/joblib_train_test/y_train.csv")
joblib.dump(X_test,"D:/ML/INTRO ML/Project/joblib_train_test/X_test.csv")
joblib.dump(y_test,"D:/ML/INTRO ML/Project/joblib_train_test/y_test.csv")
joblib.dump(df_renamed,"D:/ML/INTRO ML/Project/joblib_train_test/data.csv")

['D:/ML/INTRO ML/Project/joblib_train_test/data.csv']